### Preprocessing

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("stopwords")

In [2]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import csv
from itertools import groupby
import re


mystem = Mystem()
russian_stopwords = stopwords.words("russian")

In [3]:
def load_and_prepare(file):
    comments_file = open(file)
    comments = csv.reader(comments_file, delimiter=';')
    tpl = []

    for i in comments:
        tpl.append(i)

    flat_list = []
    for sublist in tpl:
        for item in sublist:
            if item != ' ' and item != '':
                flat_list.append(item)

    my_punctuation = [punctuation]
    my_punctuation = my_punctuation[0]
    my_punctuation += '«»—−–…'

    ready_tokens = []

    for comment in flat_list:

        comment = re.sub(r'http\S+', '', comment)
        comment = re.sub(r'\[(id|club)\d*\|.*\]', '', comment)

        token_cleared = []
        for k, g in groupby(comment):
            if k not in set(my_punctuation):
                token_cleared.extend(g)
        token = ''.join(token_cleared)

        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F680-\U0001F6FF"
                                   u"\U0001F1E0-\U0001F1FF"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001F900-\U0001F9FF"
                                   "]+", flags=re.UNICODE)
        token = emoji_pattern.sub(r'', token)

        tokens = mystem.lemmatize(token)
        tokens = [token for token in tokens if token not in russian_stopwords \
                  and token.strip() != " " and token.strip() != "" and token.strip() != "\n" \
                  and not token.isdigit()]

        if tokens:
            ready_tokens.append(tokens)

    comments_file.close()

    return ready_tokens

### Polyglot. Amount of comments with different sentiments calculation

In [7]:
from polyglot.downloader import downloader
# downloader.download('sentiment2.ru')
# downloader.download('embeddings2.ru')
# downloader.download('ner2.ru')
from polyglot.text import Text

In [8]:
comts = 'comments.txt'

comments_ready = load_and_prepare(comts)

In [11]:
amount_of_positive = 0
amount_of_negative = 0
amount_of_neutral = 0
comments_sentiments = []

for comment in comments_ready:
    comment_sentiment = 0
    phrase = ' '.join(comment)
    poly_tokens = Text(phrase, hint_language_code='ru')

    for word in poly_tokens.words:
        comment_sentiment += int(word.polarity)

    if comment_sentiment > 0:
        amount_of_positive += 1
        comments_sentiments.append('positive')
    elif comment_sentiment < 0:
        amount_of_negative += 1
        comments_sentiments.append('negative')
    else:
        amount_of_neutral += 1
        comments_sentiments.append('neutral')


print(len(comments_sentiments))

776


In [12]:
print('positive — ', amount_of_positive)
print('negative — ', amount_of_negative)
print('neutral — ', amount_of_neutral)
print("Amount of comments is equal to sum of sentiments' amounts: ",
      True if (amount_of_positive + amount_of_negative + amount_of_neutral) == len(comments_ready) else False)

positive —  316
negative —  174
neutral —  286
Amount of comments is equal to sum of sentiments' amounts:  True
